# LOFAR conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.25 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_lofar.ms")

[2024-04-11 10:30:45,243]     INFO  graphviper:  Couldn't find file metadata locally in /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox/file.download.json, checking remote API ... 


small_lofar.ms.zip:   0%|          | 0.00/9.62M [00:00<?, ?iB/s]

[2024-04-11 10:30:47,170]     INFO  graphviper:  Extracting file: small_lofar.ms.zip ... 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

ms_file = "small_lofar.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_lofar.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,small_lofar_ddi_0_intent__field_id_0,0,,0,BEAM_4,1.270615e+08,1.272324e+08,"(5, 2775, 15, 4)","[fk5, 14h00m00s, 86d00m00s]"


In [5]:
ps.get(0)

<xarray.Dataset> Size: 11MB
Dimensions:                     (time: 5, baseline_id: 2775, frequency: 15,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 11kB dask.array<chunksize=(2775,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 11kB dask.array<chunksize=(2775,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 22kB 0 1 2 ... 2773 2774
  * frequency                   (frequency) float64 120B 1.271e+08 ... 1.272e+08
  * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
  * time                        (time) float64 40B 1.52e+09 ... 1.52e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 111kB dask.array<chunksize=(5, 2775), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 832kB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 111kB dask.array<chunksize=(5, 2775), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 333kB dask.array<chunksize=(5, 2775, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 7MB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 3MB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:   {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VIS...
    ddi:           0
    intent:        
    antenna_xds:   <xarray.Dataset> Size: 14kB\nDimensions:        (antenna_i...
    pointing_xds:  <xarray.Dataset> Size: 3kB\nDimensions:        (time: 1, a...

In [6]:
ps.get(0).attrs["antenna_xds"]

<xarray.Dataset> Size: 14kB
Dimensions:        (antenna_id: 74, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 592B 0 1 2 3 4 5 6 ... 68 69 70 71 72 73
    mount          (antenna_id) <U6 2kB dask.array<chunksize=(74,), meta=np.ndarray>
    name           (antenna_id) <U9 3kB dask.array<chunksize=(74,), meta=np.ndarray>
    station        (antenna_id) <U5 1kB dask.array<chunksize=(74,), meta=np.ndarray>
    type           (antenna_id) <U12 4kB dask.array<chunksize=(74,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 592B dask.array<chunksize=(74,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 2kB dask.array<chunksize=(74, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 2kB dask.array<chunksize=(74, 3), meta=np.ndarray>